In [ ]:
# import os
# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
# os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"

In [ ]:
# stdlib
import os

environment = os.environ.get("ORCHESTRA_DEPLOYMENT_TYPE", "python")
environment

In [ ]:
# syft absolute
import syft as sy

In [ ]:
server = sy.orchestra.launch(
    name="bigquery-high",
    dev_mode=True,
    server_side_type="high",
    reset=True,
    port="8080",
    n_consumers=1,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

In [ ]:
high_client = sy.login(
    url="http://localhost:8080", email="info@openmined.org", password="changethis"
)

In [ ]:
# todo fix auth of smtp
# we should allow user pass None and skip auth
# check the implementation below and allow it work with and without auth so we can test both

In [ ]:
high_client.api.services.settings.enable_notifications(
    email_username="fake",
    email_password="fake",
    email_sender="madhava@openmined.org",
    email_server="localhost",
    email_port="1025",
)

In [ ]:
# third party
from helpers import create_user
from helpers import make_user
from helpers import save_users

In [ ]:
num_users = 2
users = []
email_disable_index = 0
for i in range(num_users):
    user = make_user()
    create_user(high_client, user)
    user.client = high_client
    if email_disable_index == i:
        user.email_disabled = True
    users.append(user)

In [ ]:
save_users(users)

In [ ]:
# users = load_users()

In [ ]:
# disable first users email
user0 = users[0]
user0.client.api.notifications.deactivate()

In [ ]:
user1 = users[1]

In [ ]:
# TODO what is this other id?
user1.client.api.user.view.id, user1.client.account.id

In [ ]:
# shouldnt this have email or something?
token = high_client.api.user.request_password_reset(user1.client.account.id)
token

In [ ]:
user1.reset_token = token

In [ ]:
new_password = user1.make_new_password()

In [ ]:
user1.client.reset_password(token=user1.reset_token, new_password=new_password)

In [ ]:
try:
    user1.client.login(email=user1.email, password=user1.password)
except Exception:
    print("Cant login with wrong password")
    assert True

In [ ]:
user1.client.login(email=user1.email, password=user1.new_password)

In [ ]:
save_users(users)

In [ ]:
assert False

In [ ]:
# !pip install aiosmtpd
# !uv pip install aiosmtpd

In [ ]:
# use to test recieving emails and move to a seperate func to call as a helper

In [ ]:
# stdlib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib

# third party
from aiosmtpd.controller import Controller

# Simple email handler class


class SimpleHandler:
    async def handle_DATA(self, server, session, envelope):
        print(f"Message from {envelope.mail_from} to {envelope.rcpt_tos}")
        print(f"Message data:\n{envelope.content.decode('utf-8', errors='replace')}")
        return "250 Message accepted for delivery"


# Start the SMTP server
handler = SimpleHandler()
controller = Controller(handler, hostname="localhost", port=1025)
controller.start()

print("SMTP server started on localhost:1025")

In [ ]:
# use to test sending emails


def send_email():
    from_address = "sender@example.com"
    to_address = "recipient@example.com"
    subject = "Test Email"
    body = "This is a test email sent to the local SMTP server."

    # Create the email message
    msg = MIMEMultipart()
    msg["From"] = from_address
    msg["To"] = to_address
    msg["Subject"] = subject
    msg.attach(MIMEText(body, "plain"))

    # SMTP server details
    smtp_server = "localhost"
    smtp_port = 1025

    # Send the email
    try:
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.sendmail(from_address, to_address, msg.as_string())
        print("Email sent successfully.")
    except Exception as e:
        print(f"Failed to send email: {e}")


# Send an email after a short delay to ensure the server is running
loop.call_later(1, send_email)